In [1]:
%load_ext autoreload
%autoreload 2

from time import time
import pandas as pd
import numpy as np
import os
import pickle
from collections import Counter, defaultdict

In [2]:
import pyterrier as pt
pt.init()

PyTerrier 0.8.0 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [3]:
from jnius import autoclass
tokeniser = autoclass("org.terrier.indexing.tokenisation.Tokeniser").getTokeniser()

In [4]:
def terrier_tokenizer(raw_utterance):
    new_utterance = " ".join(tokeniser.getTokens(raw_utterance))
    return new_utterance

In [5]:
import sys
sys.path.insert(0, "/data3/muntean/DRhard")

In [7]:
import argparse
# import subprl2_index_alls
import sys
sys.path.append("./")
import faiss
import logging
import os
import numpy as np
# import torch
from transformers import RobertaConfig
from tqdm import tqdm
from torch.utils.data.dataloader import DataLoader
from torch.utils.data.sampler import SequentialSampler

from model import RobertaDot
from dataset import (
    TextTokenIdsCache, load_rel, SubsetSeqDataset, SequenceDataset,
    single_get_collate_function
)
from retrieve_utils import (
    construct_flatindex_from_embeddings, 
    index_retrieve, convert_index_to_gpu
)
logger = logging.Logger(__name__)

/data3/muntean/DRhard/DRhard/lib/python3.8/site-packages/torch-1.7.0-py3.8-linux-x86_64.egg/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 803: system has unsupported display driver / cuda driver combination (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


# Load Star ranking

We run star inference_query.py with --no_cuda
```python ./star/inference_query.py --data_type passage --max_doc_length 256 --mode test-manual --topk 1000  --no_cuda```

```
Using mean: False
output_embedding_size 768
./data/passage/evaluate/star/test-manual-query.memmap exists, skip inference
./data/passage/evaluate/star/passages.memmap exists, skip inference
embedding shape: (38626614, 768)
(38626614,) int64
Query Num 479
100%|██████████████████████████████████████████████████████████████████████████████| 479/479 [26:17<00:00,  3.29s/it]
Elapsed Time: 1578.1s, Elapsed Time per query: 3294.5ms
```


Star output can be found in: ```/data3/muntean/DRhard/data/passage/evaluate/star/test-manual.rank.tsv```

We need to convert their ids to our ids (remapped with numbers).
```python ./cvt_back.py --input_dir ./data/passage/evaluate/star/ --preprocess_dir ./data/passage/preprocess --output_dir ./data/passage/official_runs/star --mode test-manual --dataset passage```

and then our numerical ids to the initial qids and docids, like in the qrels:

In [8]:
doc_memmap_path = "/data3/muntean/DRhard/data/passage/evaluate/star/passages.memmap"
docid_memmap_path = "/data3/muntean/DRhard/data/passage/evaluate/star/passages-id.memmap"
query_memmap_path = "/data3/muntean/DRhard/data/passage/evaluate/star/test-manual-query.memmap"
queryids_memmap_path = "/data3/muntean/DRhard/data/passage/evaluate/star/test-manual-query-id.memmap"

query_adore_memmap_path = "/data3/muntean/DRhard/data/passage/evaluate/adore-star/test-manual.qembed.memmap"

In [9]:
doc_embeddings = np.memmap(doc_memmap_path, dtype=np.float32, mode="r")
doc_ids = np.memmap(docid_memmap_path, dtype=np.int32, mode="r")
doc_embeddings = doc_embeddings.reshape(-1, 768)

query_embeddings = np.memmap(query_adore_memmap_path, dtype=np.float32, mode="r")
query_embeddings = query_embeddings.reshape(-1, 768)
query_ids = np.memmap(queryids_memmap_path, dtype=np.int32, mode="r")

In [10]:
index = construct_flatindex_from_embeddings(doc_embeddings, doc_ids)

embedding shape: (38626614, 768)
(38626614,) int64


In [13]:
# fare retireval nel indice grande e prendere top 2000 documenti
faiss.omp_set_num_threads(16) #32
nearest_neighbors = index_retrieve(index, query_embeddings, 1000, batch=32)

Query Num 479


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 479/479 [20:07<00:00,  2.52s/it]

Elapsed Time: 1207.6s, Elapsed Time per query: 2521.0ms


In [1]:
with open("/data3/muntean/conversational-cache/data/adore-star-ranking/CAST-manual-queries-adore-star-L2-ranking-top1000.raw_ids.tsv", 'w') as outputfile:
        for qid, neighbors in zip(query_ids, nearest_neighbors):
            for idx, pid in enumerate(neighbors):
                outputfile.write(f"{qid}\t{pid}\t{idx+1}\n")

NameError: name 'query_ids' is not defined

# Load id maps

In [18]:
# Load our qid and docid remapping dictionaries

# query id dict
qid_mapping_path = "/data3/muntean/DRhard/data/passage/dataset/queries.CASTmanual.QID2newID.test.tsv"
queries_df = pd.read_csv(qid_mapping_path, delimiter="\t", header=None)
print(len(queries_df))

# collection id dict
collection_mapping_path = "/data3/muntean/DRhard/data/passage/dataset/CASTcollectionPID2newID.tsv"
collection_df = pd.read_csv(collection_mapping_path, delimiter="\t", header=None)
print(len(collection_df))

479
38626614


In [19]:
# Direct dictionaries
qid2newqid_dict = dict(zip(queries_df[0], queries_df[1])) 
pid2newpid_dict = dict(zip(collection_df[0], collection_df[1])) 

In [20]:
# Create reverse dictionaries
newqid2qid_dict = dict(zip(queries_df[1], queries_df[0])) 
newpid2pid_dict = dict(zip(collection_df[1], collection_df[0]))

In [21]:
# DRhard docid and qid encoding
preprocess_dir = "/data3/muntean/DRhard/data/passage/preprocess"

pid2offset = pickle.load(open(os.path.join(preprocess_dir, "pid2offset.pickle"), 'rb'))
offset2pid = {v:k for k, v in pid2offset.items()}
qid2offset = pickle.load(open(os.path.join(preprocess_dir, f"test-manual-qid2offset.pickle"), 'rb'))
offset2qid = {v:k for k, v in qid2offset.items()}

# Remapping results

In [23]:
# results_star_dot_product_path = "/data3/muntean/conversational-cache/data/adore-star-ranking/CAST-manual-queries-adore-star-L2-ranking-top1000.raw_ids.tsv" # original queries
# results_star_dot_product_df = pd.read_csv(results_star_dot_product_path, delimiter="\t", header=None)
# print(len(results_star_dot_product_df))
# results_star_dot_product_df.head()

In [26]:
with open ('/data3/muntean/conversational-cache/data/adore-star-ranking/CAST-manual-queries-adore-star-L2-ranking-top1000.tsv', "w") as f:
    for qid, neighbors in zip(query_ids, nearest_neighbors):
            for idx, pid in enumerate(neighbors):
        
                new_qid = offset2qid[qid]
                orig_qid = newqid2qid_dict[new_qid]

                new_pid = offset2pid[pid]
                orig_pid = newpid2pid_dict[new_pid]

                f.write(f"{orig_qid}\t{orig_pid}\t{idx+1}\n")

# Starting to eval with pyterrier

In [27]:
qrel_path = "../data/CAST_qrels/qrels-docs.2019.txt"
qrels_df = pd.read_csv(qrel_path, delimiter=" ", header=None)
qrels_df[[3]] = qrels_df[[3]].astype(int)
qrels_df = qrels_df.drop([1], axis=1)
qrels_df.columns=["qid", "docno", "label"]
qrels = qrels_df

In [28]:
topics_path='../data/CAST-2019/test_manual_utterance.tsv' #manual

topics_df = pd.read_csv(topics_path, delimiter="\t", header=None)

topics_df[2] = topics_df[1].apply(lambda s: terrier_tokenizer(s))

topics_df = topics_df.drop([1], axis=1)
topics_df.columns=["qid", "query"]

topics = topics_df
topics.head()

,qid,query
0,31_1,what is throat cancer
1,31_2,is throat cancer treatable
2,31_3,tell me about lung cancer
3,31_4,what are lung cancer s symptoms
4,31_5,can lung cancer spread to the throat


In [29]:
results_path = "../data/adore-star-ranking/CAST-manual-queries-adore-star-L2-ranking-top1000.tsv"
results_df = pd.read_csv(results_path, delimiter="\t", header=None)
results_df[3] = 1000-results_df[2]
results_df.columns=["qid", "docno", "rank", "score"] # Results produced by the transformers must have “qid”, “docno”, “score”, “rank” columns.
results_df = results_df.loc[results_df['rank'] < 201]
results_df.head()

,qid,docno,rank,score
0,31_1,MARCO_3878347,1,999
1,31_1,MARCO_789620,2,998
2,31_1,MARCO_291003,3,997
3,31_1,MARCO_5625372,4,996
4,31_1,MARCO_2954451,5,995


In [30]:
%%time
pt.Experiment([results_df], topics, qrels, names=["STAR-l2"], 
              eval_metrics=["map_cut_200", "recip_rank", "recall_200", "P_3", "P_1", "ndcg_cut_3"])

CPU times: user 247 ms, sys: 170 µs, total: 247 ms
Wall time: 543 ms


,name,map_cut_200,recip_rank,recall_200,P_3,P_1,ndcg_cut_3
0,STAR-l2,0.180779,0.664051,0.441143,0.504817,0.531792,0.395627


In [ ]:
# %%time
# res_per_query = pt.Experiment([results_df], topics, qrels, names=["STAR-l2"], 
#               eval_metrics=["map", "recip_rank", "recall_200", "P_3", "P_1", "ndcg_cut_3"], perquery=True)

In [ ]:
# res_per_query

In [ ]:
# we can drop NaN rows